In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
import requests

In [2]:
# Mac users

# executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
# browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Windows users

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [4]:
news_url = 'https://mars.nasa.gov/news/'
browser.visit(news_url)

In [5]:
news_html = browser.html
soup = BeautifulSoup(news_html, 'html.parser')

# Retrieve the latest element that contains news title and news_paragraph
news_title = soup.find('div', class_='content_title').find('a').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

NASA's Mars 2020 Comes Full Circle
Aiming to pinpoint the Martian vehicle's center of gravity, engineers took NASA's 2,300-pound Mars 2020 rover for a spin in the clean room at JPL. 


# JPL Mars Space Images - Featured Image

In [6]:
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [7]:
img_html = browser.html
soup = BeautifulSoup(img_html, 'html.parser')

# Retrieve background-image url  
featured_image_url  = soup.find('article')['style'].replace('background-image: url(','').replace(');', '')[1:-1]

# Concatenate website url with scrapped route
featured_image_url = 'https://www.jpl.nasa.gov' + featured_image_url

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16711-1920x1200.jpg'

# Mars Weather

In [8]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

In [9]:
weather_html = browser.html
soup = BeautifulSoup(weather_html, 'html.parser')

# Find all elements that contain tweets
tweets = soup.find_all('div', class_='js-tweet-text-container')

# Exclude non weather related tweets 
for tweet in tweets: 
    weather_tweet = tweet.find('p').text
    
    # Find the text in a weather tweet that is not about weather and delete it
    pic_text = tweet.find('p').find('a').text
    mars_weather = weather_tweet.replace(pic_text, '')
    if 'sol' and 'low' in mars_weather:
        print(mars_weather)
        break
    else: 
        pass

InSight sol 265 (2019-08-25) low -99.4ºC (-146.9ºF) high -26.3ºC (-15.3ºF)
winds from the SSE at 5.3 m/s (12.0 mph) gusting to 16.1 m/s (35.9 mph)
pressure at 7.50 hPa


# Mars Facts

In [10]:
facts_url = 'https://space-facts.com/mars/'

# Use Pandas to parse the url
mars_facts = pd.read_html(facts_url)
mars_facts

[  Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [11]:
# Find the mars facts DataFrame in the list of DataFrames
facts_df = mars_facts[1]
# Assign the columns names
facts_df.columns = ['Description', 'Value']
facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
# Convert to html
html_table = facts_df.to_html(header=False, index=False, classes="dataframe table table-striped")
print(html_table)

<table border="1" class="dataframe dataframe table table-striped">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


# Mars Hemispheres

In [13]:
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

In [14]:
hemispheres_html = browser.html
soup = BeautifulSoup(hemispheres_html, 'html.parser')

# Retreive all elements that contain mars hemispheres information
imgs = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main url
main_url = 'https://astrogeology.usgs.gov'

for img in imgs: 
    # Find and store the title 
    title = img.find('h3').text
    # Find partial url to the full image
    partial_url = img.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image  
    browser.visit(main_url + partial_url)
    img_html = browser.html
    soup = BeautifulSoup(img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]